In [2]:
import pandas as pd

In [3]:
name = 'SBIN'

In [3]:
def files(name: str, no: int):
    df = pd.read_parquet(f'order_{no}.parquet')
    df = df[df['Symbol'] == name]
    return len(df) > 0

def files_seq(name: str):
    i = 1
    res = []
    while True:
        try:
            if files(name, i):
                res.append(i)
            i += 1
        except FileNotFoundError:
            break

        if i > 5:
            break
    return res

name = 'SBIN'

print(files_seq(name))





[4, 5]


In [5]:
df1 = pd.read_parquet('order_4.parquet')
df2 = pd.read_parquet('order_5.parquet')

In [ ]:
df = pd.concat([df1, df2])


In [ ]:
print(len(df) == len(df1) + len(df2))

True


In [ ]:
df = df[df['Symbol'] == name]

In [ ]:
df.head()

,Timestamp,Side,Price,Volume_Disclosed,Price_Type,Symbol,Original_Qty,Order Number,IOC Flag,Activity Type
22392942,79631194614632,B,258.85,0,Limit,SBIN,1,1300000000002815,N,1
22392956,79631194618744,S,256.00,0,Limit,SBIN,1,1300000000002919,N,1
22393004,79631194632638,S,260.00,0,Limit,SBIN,200,1300000000003115,N,1
22393135,79631194641354,B,252.00,0,Limit,SBIN,22,1300000000003281,N,1
22393282,79631194673336,S,265.00,0,Limit,SBIN,300,1300000000003440,N,1


In [ ]:
can = pd.read_parquet('cancelled.parquet')

In [5]:
can = can.rename(columns = {'TimeStamp' : 'Timestamp'})

In [6]:
can.head()

,Timestamp,Order Number,IOC Flag,Activity Type
0,79631194995693,1000000000007281,N,3
1,79631195013045,1000000000005540,N,3
2,79631195115036,1000000000006571,N,3
3,79631195116810,1000000000007282,N,3
4,79631195125870,1000000000007860,N,3


In [ ]:
can_filtered = can[can['Order Number'].isin(data['Order Number'])]

In [8]:
can_filtered.head()


,Timestamp,Order Number,IOC Flag,Activity Type
2068857,79631195649261,1300000000012205,N,3
2068874,79631196044314,1300000000013284,N,3
2068877,79631196200750,1300000000015708,N,3
2068892,79631196768692,1300000000018763,N,3
2068895,79631196807554,1300000000019109,N,3


In [9]:
len(can_filtered)

121312

In [ ]:
data = pd.concat([df, can_filtered], ignore_index=True)


In [11]:
print(len(data) == len(df) + len(can_filtered))

True


In [12]:
data.head(100000000)

,Timestamp,Side,Price,Volume_Disclosed,Price_Type,Symbol,Original_Qty,Order Number,IOC Flag,Activity Type
0,79631194614632,B,258.85,0.0,Limit,SBIN,1.0,1300000000002815,N,1
1,79631194618744,S,256.00,0.0,Limit,SBIN,1.0,1300000000002919,N,1
2,79631194632638,S,260.00,0.0,Limit,SBIN,200.0,1300000000003115,N,1
3,79631194641354,B,252.00,0.0,Limit,SBIN,22.0,1300000000003281,N,1
4,79631194673336,S,265.00,0.0,Limit,SBIN,300.0,1300000000003440,N,1
...,...,...,...,...,...,...,...,...,...,...
1548525,79632823944584,NaN,NaN,NaN,NaN,NaN,NaN,1300000001412980,N,3
1548526,79632824273477,NaN,NaN,NaN,NaN,NaN,NaN,1300000000773434,N,3
1548527,79632828544726,NaN,NaN,NaN,NaN,NaN,NaN,1300000003495314,N,3
1548528,79632832063279,NaN,NaN,NaN,NaN,NaN,NaN,1300000003668735,N,3


In [13]:
data.to_parquet(f'{name}_order.parquet')